In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

data = pd.read_csv("aefi.csv")

In [10]:
data.head(5)

,date,vaxtype,daily_total,daily_serious_npra,daily_nonserious,daily_nonserious_npra,daily_nonserious_mysj_dose1,daily_nonserious_mysj_dose2,d1_site_pain,d1_site_swelling,...,d2_site_redness,d2_tiredness,d2_headache,d2_muscle_pain,d2_joint_pain,d2_weakness,d2_fever,d2_vomiting,d2_chills,d2_rash
0,2021-01-13,astrazeneca,5,0,5,5,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2021-02-25,pfizer,1,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2021-02-26,pfizer,2,0,2,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2021-02-27,pfizer,4,0,4,4,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2021-02-28,pfizer,12,1,11,11,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
import pandas as pd

# Read the CSV file
data = pd.read_csv("aefi.csv")

# Convert the 'date' column to datetime, allowing mixed formats
data['date'] = pd.to_datetime(data['date'], dayfirst=True, errors='coerce')

# Define the date range
start_date = '2021-02-01'
end_date = '2021-07-01'

# Filter the data within the specified date range
filtered_data = data[(data['date'] >= start_date) & (data['date'] <= end_date)]

# Display the filtered data
print(filtered_data)


          date      vaxtype  daily_total  daily_serious_npra  \
1   2021-02-25       pfizer            1                   0   
2   2021-02-26       pfizer            2                   0   
3   2021-02-27       pfizer            4                   0   
4   2021-02-28       pfizer           12                   1   
5   2021-03-01       pfizer           33                   4   
..         ...          ...          ...                 ...   
275 2021-06-30       pfizer         1656                   7   
276 2021-06-30      sinovac         1352                  14   
277 2021-07-01  astrazeneca          293                   3   
278 2021-07-01       pfizer          842                   6   
279 2021-07-01      sinovac         1179                   3   

     daily_nonserious  daily_nonserious_npra  daily_nonserious_mysj_dose1  \
1                   1                      1                            0   
2                   2                      2                            0   


C:\Users\cheng\AppData\Local\Temp\ipykernel_11384\1297870269.py:7: UserWarning: Parsing dates in %Y-%m-%d format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  data['date'] = pd.to_datetime(data['date'], dayfirst=True, errors='coerce')
